#All


## Set up

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.4.1


In [ ]:
print("Installing dependencies...")
%tensorflow_version 2.x
# !pip install --user --upgrade tensorflow-probability
# !pip install -U tf-nightly-gpu
# !pip install -U tensorflow-gcs-config==2.1.2
!pip install -q t5==0.7.1

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import gin
import t5

Installing dependencies...
     |████████████████████████████████| 174kB 6.8MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 3.4MB 7.7MB/s 
     |████████████████████████████████| 3.9MB 42.2MB/s 
     |████████████████████████████████| 1.2MB 45.8MB/s 
     |████████████████████████████████| 2.1MB 44.4MB/s 
     |████████████████████████████████| 368kB 42.6MB/s 
     |████████████████████████████████| 3.3MB 43.8MB/s 
     |████████████████████████████████| 901kB 42.5MB/s 


In [ ]:
ON_CLOUD = True

if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "v3-8"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU zdetection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)


Setting up GCS access...
Running on TPU: grpc://10.110.233.242:8470
Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import gin
import subprocess
gin.parse_config_file(
        'gs://t5-data/pretrained_models/base/operative_config.gin'
    )


INFO:root:system_path_file_exists:gs://t5-data/pretrained_models/base/operative_config.gin
ERROR:root:Path not found: gs://t5-data/pretrained_models/base/operative_config.gin


ParsedConfigFileIncludesAndImports(filename='gs://t5-data/pretrained_models/base/operative_config.gin', imports=['t5.models.mesh_transformer', 't5.data.sentencepiece_vocabulary', 'mesh_tensorflow.optimize', 'mesh_tensorflow.transformer.dataset', 'mesh_tensorflow.transformer.learning_rate_schedules', 'mesh_tensorflow.transformer.t2t_vocabulary', 'mesh_tensorflow.transformer.transformer_layers', 'mesh_tensorflow.transformer.utils'], includes=[])

## dumping dataset

In [ ]:
from random import shuffle

def dumping_dataset(split, shuffle_files = False):
    del shuffle_files
    
    train_files = []

    train_files.extend(list(map(lambda x: x.strip(), subprocess.run(['gsutil', 'ls', f'gs://cotext/data/github_repos/sample_content/python/train/*.tsv'], stdout=subprocess.PIPE).stdout.splitlines())))
    train_files.extend(list(map(lambda x: x.strip(), subprocess.run(['gsutil', 'ls', f'gs://cotext/data/github_repos/sample_content/java/train/*.tsv'], stdout=subprocess.PIPE).stdout.splitlines())))

    shuffle(train_files)

    print(train_files[0])

    ds = tf.data.TextLineDataset(
       train_files
    )
    ds = ds.map(lambda *ex: dict(zip(['title', 'text'], ['None',ex[0]])))

    return ds

In [ ]:

t5.data.TaskRegistry.remove('dumping_dataset')
t5.data.TaskRegistry.add(
    'dumping_dataset',
    dataset_fn = dumping_dataset,
    splits = ['train'],
    text_preprocessor =  functools.partial(
        t5.data.preprocessors.rekey,
        key_map = {'inputs': None, 'targets': 'text'},
    ),
    token_preprocessor = t5.data.preprocessors.unsupervised,
    metric_fns = [],
)


In [ ]:
t5.data.MixtureRegistry.remove('all_codeT5')
t5.data.MixtureRegistry.add(
    'all_codeT5',
    [
        'dumping_dataset',
    ],
    default_rate = 1.0,
)

## Pre train

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)


MODEL_SIZE = 'base'
model_parallelism, train_batch_size, keep_checkpoint_max = {
    'small': (1, 256, 16),
    'base': (2, 128, 8),
    'large': (8, 64, 4),
    '3B': (8, 16, 1),
    '11B': (8, 16, 1),
}[MODEL_SIZE]

In [ ]:
if ON_CLOUD:
  %reload_ext tensorboard
  import tensorboard as tb
tb.notebook.start("--logdir " + 'gs://t5_training/models/code/code_all_v1/base')

In [ ]:
model = t5.models.MtfModel(
  model_dir = 'gs://t5_training/models/code/code_all_v1/base',
  tpu = TPU_ADDRESS,
  tpu_topology = TPU_TOPOLOGY,
  model_parallelism = model_parallelism,
  batch_size = train_batch_size,
  sequence_length = {'inputs': 1024, 'targets': 1024},
  learning_rate_schedule = 0.001,
  save_checkpoints_steps = 1000,
  keep_checkpoint_max = 5,
  iterations_per_loop = 100,
)

model.train(mixture_or_task_name = 'all_codeT5', steps = 1000000+200000)